In [2]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB 435.7 kB/s eta 0:01:28
   ---------------------------------------- 0.1/38.1 MB 939.4 kB/s eta 0:00:41
   ---------------------------------------- 0.3/38.1 MB 2.0 MB/s eta 0:00:20
    --------------------------------------- 0.5/38.1 MB 2.6 MB/s eta 0:00:15
   - -------------------------------------- 1.0/38.1 MB 4.1 MB/s eta 0:00:10
   - -------------------------------------- 1.2/38.1 MB 4.0 MB/s eta 0:00:10
   - -------------------------------------- 1.4/38.1 MB 4.1 MB/s eta 0:00:09
   - -------------------------------------- 1.4/38.1 MB 4.1 MB/s eta 0:00:09
   - --------------

In [11]:
!pip install pillow --user
!pip install pytesseract --user

In [12]:
!pip install spacy --user

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/ad/ca/1f6d0446d8ed55290ca49db6416c4faeeca579d127f379a2e360f179b69c/spacy-3.7.1-cp311-cp311-win_amd64.whl.metadata
  Using cached spacy-3.7.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Using cached murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata (2.0 kB)
  Obtaining dependency information for preshed<3.1.0,>=3.0.2 

In [13]:
import numpy as np
import pandas as pd
import cv2
import PIL #Python Image Library
import pytesseract
import spacy

In [ ]:
img_cv2 = cv2.imread('./BusinessCardData/Selected/052.jpeg')
cv2.imshow('Business Card',img_cv2)
cv2.waitKey(0) #Manually Closing the open image window
cv2.destroyAllWindows() #once the waitkey is over, need to destroy all the windows

In [ ]:
img_pil = PIL.Image.open('./BusinessCardData/Selected/052.jpeg')

In [ ]:
type(img_pil) #outputs as an object

In [ ]:
img_pil

In [ ]:
type(img_cv2)

## Text from Image

In [ ]:
text_cv = pytesseract.image_to_string(img_cv2)

In [ ]:
print(text_cv)

In [ ]:
text_pl = pytesseract.image_to_string(img_pil)

In [ ]:
print(text_pl)

## Image to Data

In [ ]:
data = pytesseract.image_to_data(img_cv2)

In [ ]:
print(data)

In [ ]:
data

In [ ]:
data.split('\n')

In [ ]:
list(map(lambda x:x.split('\t'),['level\tpage_num\tblock_num\tpar_num\tline_num\tword_num\tleft\ttop\twidth\theight\tconf\ttext']))

In [ ]:
dataList = list(map(lambda x:x.split('\t'),data.split('\n')))
dataList

In [ ]:
df = pd.DataFrame(dataList[1:],columns=dataList[0])

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.dropna(inplace=True) # drop the missing values in rows
columns_int = ['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num','left', 'top', 'width', 'height','conf']
# df[columns_int] = df[columns_int].astype(int)
df[columns_int] = df[columns_int].astype(float).round(0).astype(int)

In [ ]:
df.info()

In [ ]:
image = img_cv2.copy()
level = 'words'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
    #print(l,x,y,w,h,c)
    if level == 'page':
        if l == 1:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,0),2)
        else:
            continue
    elif level == 'block':
        if l == 2:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)  #BGR format for cv2
        else:
            continue
    elif level == 'para':
        if l == 3:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)  #BGR format for cv2
        else:
            continue
    elif level == 'line':
        if l == 4:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)  #BGR format for cv2
        else:
            continue
    
    elif level == 'words':
        if l == 5:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)  #BGR format for cv2
            cv2.putText(image,txt,(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
cv2.imshow("Bounding Box:",image)
cv2.waitKey()
cv2.destroyAllWindows()